In [114]:
def get_df_from_json(data_url, record_path=['Data']):
    content = requests.get(data_url).json()
    df_all = pd.json_normalize(content,record_path=record_path)
    
    return df_all

In [115]:
# Scrape webpage
import io
import requests
import pandas as pd
from datetime import date, timedelta

# Start date and end date
today = date.today()
start_date = '2020-01-01'
end_date = date.today() - timedelta(days=1)

In [116]:
# Get river data
river_data_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=35946&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
flow_df_all = get_df_from_json(river_data_url)
flow_df = flow_df_all[["TimeStamp", "Value"]]
flow_df = flow_df.rename(columns={'Value': 'FlowRate'})
flow_df['TimeStamp']= pd.to_datetime(flow_df['TimeStamp'])

print(flow_df.shape)
print(flow_df.head())

(293741, 2)
                  TimeStamp   FlowRate
0 2022-10-17 00:00:00+00:00  39.085441
1 2022-10-16 23:55:00+00:00  38.851989
2 2022-10-16 23:50:00+00:00  39.386349
3 2022-10-16 23:45:00+00:00  39.152236
4 2022-10-16 23:40:00+00:00  38.851989


In [117]:
# Get Lake levels
lake_level_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=32419&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
lake_level_df_all = get_df_from_json(lake_level_url)
lake_level_df = lake_level_df_all[["TimeStamp", "Value"]]
lake_level_df = lake_level_df.rename(columns={'Value': 'LakeLevel'})
lake_level_df['TimeStamp']= pd.to_datetime(lake_level_df['TimeStamp'])
print(lake_level_df.shape)
print(lake_level_df.head())


(293824, 2)
                  TimeStamp LakeLevel
0 2022-10-17 00:00:00+00:00    279.19
1 2022-10-16 23:55:00+00:00   279.186
2 2022-10-16 23:50:00+00:00   279.178
3 2022-10-16 23:45:00+00:00   279.178
4 2022-10-16 23:40:00+00:00   279.182


In [122]:
# Get gate levels
#Gate 1
gate_levels_url_1 = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=38970&sort=TimeStamp-desc&page=1&pageSize=100&group=&filter=&interval=Latest&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
gate_1_df = get_df_from_json(gate_levels_url_1)
gate_1_df = gate_1_df[["TimeStamp", "Value"]]
gate_1_df = gate_1_df.rename(columns={"Value": "Gate1"})

# Gate 2
gate_levels_url_2 = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=38973&sort=TimeStamp-desc&page=1&pageSize=100&group=&filter=&interval=Latest&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
gate_2_df = get_df_from_json(gate_levels_url_2)
gate_2_df = gate_2_df[["TimeStamp", "Value"]]
gate_2_df = gate_2_df.rename(columns={"Value": "Gate2"})

# Gate 3
gate_levels_url_3 = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=38972&sort=TimeStamp-desc&page=1&pageSize=100&group=&filter=&interval=Latest&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
gate_3_df = get_df_from_json(gate_levels_url_3)
gate_3_df = gate_3_df[["TimeStamp", "Value"]]
gate_3_df = gate_3_df.rename(columns={"Value": "Gate3"})

# Concatenate into single dataframe
gate_levels_df_temp['TimeStamp']= pd.to_datetime(gate_levels_df_temp['TimeStamp'])
gate_levels_df_temp = pd.merge(gate_1_df, gate_2_df, on='TimeStamp')
gate_levels_df = pd.merge(gate_levels_df_temp, gate_3_df, on='TimeStamp', how='left')

print(gate_levels_df.head())

              TimeStamp   Gate1   Gate2   Gate3
0  2022-10-18T10:40:00Z  1500.0  1500.0  1500.0
1  2022-10-18T10:35:00Z  1500.0  1500.0  1500.0
2  2022-10-18T10:30:00Z  1500.0  1500.0  1500.0
3  2022-10-18T10:25:00Z  1500.0  1500.0  1500.0
4  2022-10-18T10:20:00Z  1500.0  1500.0  1500.0


In [123]:
# Get rainfall at Lake Rotoiti

rainfall_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=47928&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
rainfall_df_all = get_df_from_json(rainfall_url)
rainfall_df = rainfall_df_all[["TimeStamp", "Value"]]
rainfall_df = rainfall_df.rename(columns={'Value': 'Rainfall'})
rainfall_df['TimeStamp']= pd.to_datetime(rainfall_df['TimeStamp'])
print(rainfall_df.head())
print(rainfall_df.shape)

                  TimeStamp  Rainfall
0 2022-10-17 00:00:00+00:00       0.0
1 2022-10-16 23:00:00+00:00       0.0
2 2022-10-16 22:00:00+00:00       0.0
3 2022-10-16 21:00:00+00:00       0.0
4 2022-10-16 20:00:00+00:00       0.0
(24481, 2)


In [124]:
# Merge datasets into one dataframe
df_temp = pd.merge(lake_level_df, flow_df, on='TimeStamp')
df_temp = pd.merge(df_temp, gate_levels_df, on='TimeStamp')
df_combined = pd.merge(df_temp, rainfall_df, on='TimeStamp', how='left')

#Set index
df_combined['TimeStamp'] = pd.to_datetime(df_combined['TimeStamp'])
df_combined.set_index('TimeStamp', inplace=True)

#Remove rows for which we don't have rainfall data
df_river_data = df_combined.dropna()

#Sort by timestamp descending
df_river_data = df_river_data.sort_values('TimeStamp')

print (df_river_data)

ValueError: You are trying to merge on datetime64[ns, UTC] and object columns. If you wish to proceed you should use pd.concat

In [ ]:
#Save to csv
df_river_data.to_csv('kaituna_data.csv')